# Hong Kong District Clustering by Venues from Foursquare

In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from urllib.request import urlopen
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests 
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium 
print("Libraries has been imported!")

Libraries has been imported!


In [2]:
#specifying the URL to scrape data from
url = "https://en.wikipedia.org/wiki/Districts_of_Hong_Kong"

# open the url and put it in a variable
page = urllib.request.urlopen(url)

# parse the HTML from our URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(page, "lxml")

#Checking the underlying HTML source code
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Districts of Hong Kong - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XsC76wpAIH4AA7Q3VBkAAAAR","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Districts_of_Hong_Kong","wgTitle":"Districts of Hong Kong","wgCurRevisionId":953006501,"wgRevisionId":953006501,"wgArticleId":151994,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Webarchive template wayback links","EngvarB from July 2014","Use dmy dates from July 2014","Articles containing Chinese-language text","P

In [3]:
all_tables = soup.find_all("table")
all_tables

[<table class="infobox vevent" style="width:22em;float: right; width: 250px; font-size: 90%; text-align: left; border-spacing: 3px;"><tbody><tr><th class="summary" colspan="2" style="text-align:center;font-size:125%;font-weight:bold;font-size: 130%; background-color: #F0F0F0; vertical-align: middle">Districts of Hong Kong<br/><style data-mw-deduplicate="TemplateStyles:r886047488">.mw-parser-output .nobold{font-weight:normal}</style><span class="nobold"><span lang="zh-Hant" title="Chinese language text">香港地區</span></span></th></tr><tr><td colspan="2" style="text-align:center"><a class="image" href="/wiki/File:Map_of_Hong_Kong_18_Districts_en.svg"><img alt="Map of Hong Kong 18 Districts en.svg" data-file-height="1083" data-file-width="1454" decoding="async" height="186" src="//upload.wikimedia.org/wikipedia/commons/thumb/f/fe/Map_of_Hong_Kong_18_Districts_en.svg/250px-Map_of_Hong_Kong_18_Districts_en.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/f/fe/Map_of_Hong_Kong_18

In [4]:
right_table=soup.find('table', class_='wikitable sortable')
right_table

<table class="wikitable sortable">
<tbody><tr>
<th>District
</th>
<th><a href="/wiki/Traditional_Chinese_characters" title="Traditional Chinese characters">Chinese</a>
</th>
<th>Population<sup class="noprint Inline-Template" style="white-space:nowrap;">[<i><a href="/wiki/Wikipedia:Manual_of_Style/Dates_and_numbers#Chronological_items" title="Wikipedia:Manual of Style/Dates and numbers"><span title="The time period mentioned near this tag is ambiguous. (December 2019)">when?</span></a></i>]</sup> <sup class="reference" id="cite_ref-6"><a href="#cite_note-6">[6]</a></sup>
</th>
<th>Area<br/>(km²)
</th>
<th>Density<br/>(/km²)
</th>
<th>Region
</th></tr>
<tr>
<td><a href="/wiki/Central_and_Western_District" title="Central and Western District"><span class="nowrap">Central and Western</span></a></td>
<td><span lang="zh-HK" title="Chinese language text">中西區</span></td>
<td align="right">244,600</td>
<td align="right">12.44</td>
<td align="right">19,983.92</td>
<td><span class="nowrap"><a hre

In [5]:
A=[]
B=[]
C=[]
D=[]
E=[]
F=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==6:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))
        D.append(cells[3].find(text=True))
        E.append(cells[4].find(text=True))
        F.append(cells[5].find(text=True))
        
df=pd.DataFrame(A,columns=['District'])
df['Chinese']=B
df['Population']=C
df['Area']=D
df['Density']=E
df['Region']=F
df

,District,Chinese,Population,Area,Density,Region
0,Central and Western,中西區,"244,600",12.44,"19,983.92",Hong Kong Island
1,Eastern,東區,"574,500",18.56,"31,217.67",Hong Kong Island
2,Southern,南區,"269,200",38.85,"6,962.68",Hong Kong Island
3,Wan Chai,灣仔區,"150,900",9.83,"15,300.10",Hong Kong Island
4,Sham Shui Po,深水埗區,"390,600",9.35,"41,529.41",Kowloon
5,Kowloon City,九龍城區,"405,400",10.02,"40,194.70",Kowloon
6,Kwun Tong,觀塘區,"641,100",11.27,"56,779.05",Kowloon
7,Wong Tai Sin,黃大仙區,"426,200",9.30,"45,645.16",Kowloon
8,Yau Tsim Mong,油尖旺區,"318,100",6.99,"44,864.09",Kowloon
9,Islands,離島區,"146,900",175.12,825.14,New Territories


Geocode does not have specific coordinates for the 18 Districts. As a result, I have resorted to using GPS coordinates from GeoHack and incorporating the data manually to combine. In addition, I have also elinminated the New Territories given districts in those areas in general tends to be old and far away from the central business district of Hong Kong.

In [6]:
#removing New Territories Districts
df = df[df.Region != 'New Territories']
df

,District,Chinese,Population,Area,Density,Region
0,Central and Western,中西區,"244,600",12.44,"19,983.92",Hong Kong Island
1,Eastern,東區,"574,500",18.56,"31,217.67",Hong Kong Island
2,Southern,南區,"269,200",38.85,"6,962.68",Hong Kong Island
3,Wan Chai,灣仔區,"150,900",9.83,"15,300.10",Hong Kong Island
4,Sham Shui Po,深水埗區,"390,600",9.35,"41,529.41",Kowloon
5,Kowloon City,九龍城區,"405,400",10.02,"40,194.70",Kowloon
6,Kwun Tong,觀塘區,"641,100",11.27,"56,779.05",Kowloon
7,Wong Tai Sin,黃大仙區,"426,200",9.30,"45,645.16",Kowloon
8,Yau Tsim Mong,油尖旺區,"318,100",6.99,"44,864.09",Kowloon


In [7]:
latitudes = [22.2833322, 22.28411, 22.24725, 22.27968, 22.33074, 22.3282, 22.31326, 22.33353, 22.32138]
longitudes = [114.1499994, 114.22414, 114.15884, 114.17168, 114.1622, 114.19155, 114.22581, 114.19686, 114.1726]
df['Latitude'] = pd.Series(latitudes, index=df.index)
df['Longitude'] = pd.Series(longitudes, index=df.index)
df.head()

D:\Python\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Python\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,District,Chinese,Population,Area,Density,Region,Latitude,Longitude
0,Central and Western,中西區,"244,600",12.44,"19,983.92",Hong Kong Island,22.283332,114.149999
1,Eastern,東區,"574,500",18.56,"31,217.67",Hong Kong Island,22.284110,114.224140
2,Southern,南區,"269,200",38.85,"6,962.68",Hong Kong Island,22.247250,114.158840
3,Wan Chai,灣仔區,"150,900",9.83,"15,300.10",Hong Kong Island,22.279680,114.171680
4,Sham Shui Po,深水埗區,"390,600",9.35,"41,529.41",Kowloon,22.330740,114.162200


Importing FourSquare Credentials and Calls

In [8]:
CLIENT_ID = '5U55EJL5TGVU0SRKJ05II0HVI515EHWKVO54HKOVZXGAG2PE' # your Foursquare ID
CLIENT_SECRET = 'ZBMLZUWPNE3VI2LB155C2UQJDRGPYKKV34MFKMGF21JOU2NV' # your Foursquare Secret
VERSION = '20200515' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

LIMIT = 100
radius = 500

Your credentails:
CLIENT_ID: 5U55EJL5TGVU0SRKJ05II0HVI515EHWKVO54HKOVZXGAG2PE
CLIENT_SECRET:ZBMLZUWPNE3VI2LB155C2UQJDRGPYKKV34MFKMGF21JOU2NV


In [9]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'District Latitude', 
                  'District Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [10]:
hk_venues = getNearbyVenues(names=df['District'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Central and Western
Eastern
Southern
Wan Chai
Sham Shui Po
Kowloon City
Kwun Tong
Wong Tai Sin
Yau Tsim Mong


In [11]:
hk_venues.shape #checking venues size on the dataframe

(443, 7)

In [12]:
hk_venues.head()

,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Central and Western,22.283332,114.149999,PMQ (元創方),22.283298,114.151971,Arts & Crafts Store
1,Central and Western,22.283332,114.149999,The Old Man,22.282770,114.151774,Bar
2,Central and Western,22.283332,114.149999,Blue Supreme,22.285148,114.149385,Bar
3,Central and Western,22.283332,114.149999,Teakha (茶·家),22.284506,114.148407,Tea Room
4,Central and Western,22.283332,114.149999,Craftissimo,22.284589,114.148293,Beer Store


In [13]:
hk_venues_pv = pd.pivot_table(hk_venues,index="District",values="Venue Category",aggfunc='count')
hk_venues_pv

,Venue Category
District,
Central and Western,96
Eastern,12
Kowloon City,68
Kwun Tong,61
Sham Shui Po,44
Southern,18
Wan Chai,60
Wong Tai Sin,13
Yau Tsim Mong,71


Eastern, Southern and Wong Tai Sin has noticeablely less entries compare to other places.

In [14]:
# one hot encoding
hk_onehot = pd.get_dummies(hk_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
hk_onehot['District'] = hk_venues['District'] 

# move neighborhood column to the first column
fixed_columns = [hk_onehot.columns[-1]] + list(hk_onehot.columns[:-1])
hk_onehot = hk_onehot[fixed_columns]

hk_onehot.head()

,District,African Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bakery,Bar,Beer Bar,Beer Store,Beijing Restaurant,Bistro,Bookstore,Brazilian Restaurant,Bubble Tea Shop,Building,Burger Joint,Bus Stop,Café,Cantonese Restaurant,Cha Chaan Teng,Chinese Breakfast Place,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Cosmetics Shop,Creperie,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Donburi Restaurant,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food Court,French Restaurant,Furniture / Home Store,Garden,Gastropub,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,History Museum,Hobby Shop,Hong Kong Restaurant,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lebanese Restaurant,Lounge,Malay Restaurant,Market,Massage Studio,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Multiplex,Noodle House,Outdoor Supply Store,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Perfume Shop,Pizza Place,Portuguese Restaurant,Ramen Restaurant,Restaurant,Sake Bar,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Toy / Game Store,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio
0,Central and Western,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Central and Western,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Central and Western,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Central and Western,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,Central and Western,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
hk_grouped = hk_onehot.groupby('District').mean().reset_index()
hk_grouped

,District,African Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bakery,Bar,Beer Bar,Beer Store,Beijing Restaurant,Bistro,Bookstore,Brazilian Restaurant,Bubble Tea Shop,Building,Burger Joint,Bus Stop,Café,Cantonese Restaurant,Cha Chaan Teng,Chinese Breakfast Place,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Cosmetics Shop,Creperie,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Donburi Restaurant,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food Court,French Restaurant,Furniture / Home Store,Garden,Gastropub,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,History Museum,Hobby Shop,Hong Kong Restaurant,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lebanese Restaurant,Lounge,Malay Restaurant,Market,Massage Studio,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Multiplex,Noodle House,Outdoor Supply Store,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Perfume Shop,Pizza Place,Portuguese Restaurant,Ramen Restaurant,Restaurant,Sake Bar,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Toy / Game Store,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio
0,Central and Western,0.000000,0.010417,0.010417,0.010417,0.000000,0.000000,0.010417,0.000000,0.020833,0.031250,0.010417,0.020833,0.000000,0.010417,0.000000,0.010417,0.000000,0.000000,0.000000,0.000000,0.072917,0.010417,0.000000,0.000000,0.020833,0.000000,0.000000,0.020833,0.072917,0.000000,0.000000,0.010417,0.000000,0.000000,0.000000,0.000000,0.010417,0.000000,0.000000,0.010417,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052083,0.000000,0.010417,0.010417,0.010417,0.010417,0.000000,0.010417,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010417,0.000000,0.000000,0.000000,0.010417,0.010417,0.000000,0.062500,0.072917,0.000000,0.010417,0.010417,0.020833,0.000000,0.000000,0.000000,0.000000,0.010417,0.010417,0.010417,0.000000,0.010417,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010417,0.020833,0.031250,0.010417,0.010417,0.010417,0.000000,0.000000,0.000000,0.010417,0.000000,0.010417,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010417,0.010417,0.020833,0.000000,0.020833,0.041667,0.010417,0.03125,0.010417,0.000000
1,Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
2,Kowloon City,0.000000,0.000000,0.000000,0.000000,0.044118,0.000000,0.00

In [16]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [17]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['District']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

columns

['District',
 '1st Most Common Venue',
 '2nd Most Common Venue',
 '3rd Most Common Venue',
 '4th Most Common Venue',
 '5th Most Common Venue',
 '6th Most Common Venue',
 '7th Most Common Venue',
 '8th Most Common Venue',
 '9th Most Common Venue',
 '10th Most Common Venue']

In [18]:
# create a new dataframe
districts_venues_sorted = pd.DataFrame(columns=columns)
districts_venues_sorted['District'] = hk_grouped['District']

for ind in np.arange(hk_grouped.shape[0]):
    districts_venues_sorted.iloc[ind, 1:] = return_most_common_venues(hk_grouped.iloc[ind, :], num_top_venues)

districts_venues_sorted.head()

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central and Western,Japanese Restaurant,Coffee Shop,Café,Italian Restaurant,French Restaurant,Vietnamese Restaurant,Wine Bar,Bar,Restaurant,Bakery
1,Eastern,Park,Chinese Restaurant,Hong Kong Restaurant,Cantonese Restaurant,Indian Restaurant,French Restaurant,Japanese Restaurant,Fish & Chips Shop,Restaurant,Harbor / Marina
2,Kowloon City,Thai Restaurant,Dessert Shop,Chinese Restaurant,Coffee Shop,Café,Asian Restaurant,Halal Restaurant,Noodle House,Fast Food Restaurant,Cha Chaan Teng
3,Kwun Tong,Chinese Restaurant,Café,Coffee Shop,Cha Chaan Teng,Fast Food Restaurant,Japanese Restaurant,Restaurant,Cantonese Restaurant,Sushi Restaurant,Department Store
4,Sham Shui Po,Noodle House,Chinese Restaurant,Dessert Shop,Hong Kong Restaurant,Italian Restaurant,Shopping Mall,Snack Place,Electronics Store,Market,Malay Restaurant


In [19]:
# set number of clusters
kclusters = 3

hk_grouped_clustering = hk_grouped.drop('District', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(hk_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100]

array([0, 1, 2, 0, 0, 0, 0, 0, 0])

In [20]:
# add clustering labels
districts_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [21]:
hk_merged = df

#merging to add long/lat values
hk_merged = hk_merged.join(districts_venues_sorted.set_index('District'), on='District')

hk_merged.head() 

,District,Chinese,Population,Area,Density,Region,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central and Western,中西區,"244,600",12.44,"19,983.92",Hong Kong Island,22.283332,114.149999,0,Japanese Restaurant,Coffee Shop,Café,Italian Restaurant,French Restaurant,Vietnamese Restaurant,Wine Bar,Bar,Restaurant,Bakery
1,Eastern,東區,"574,500",18.56,"31,217.67",Hong Kong Island,22.284110,114.224140,1,Park,Chinese Restaurant,Hong Kong Restaurant,Cantonese Restaurant,Indian Restaurant,French Restaurant,Japanese Restaurant,Fish & Chips Shop,Restaurant,Harbor / Marina
2,Southern,南區,"269,200",38.85,"6,962.68",Hong Kong Island,22.247250,114.158840,0,Cha Chaan Teng,Fast Food Restaurant,Hong Kong Restaurant,Park,Athletics & Sports,BBQ Joint,Chinese Restaurant,Coffee Shop,Dessert Shop,Hotel
3,Wan Chai,灣仔區,"150,900",9.83,"15,300.10",Hong Kong Island,22.279680,114.171680,0,Coffee Shop,Chinese Restaurant,Hotel,Japanese Restaurant,Cha Chaan Teng,Middle Eastern Restaurant,Pizza Place,Clothing Store,Hong Kong Restaurant,Cantonese Restaurant
4,Sham Shui Po,深水埗區,"390,600",9.35,"41,529.41",Kowloon,22.330740,114.162200,0,Noodle House,Chinese Restaurant,Dessert Shop,Hong Kong Restaurant,Italian Restaurant,Shopping Mall,Snack Place,Electronics Store,Market,Malay Restaurant


In [22]:
# create map showing clusters
latitude = 22.3
longitude = 114.2
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(hk_merged['Latitude'], hk_merged['Longitude'], hk_merged['District'], hk_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [23]:
#1st Cluster
hk_merged.loc[hk_merged['Cluster Labels'] == 0, hk_merged.columns[[1] + list(range(3, hk_merged.shape[1]))]]

,Chinese,Area,Density,Region,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,中西區,12.44,"19,983.92",Hong Kong Island,22.283332,114.149999,0,Japanese Restaurant,Coffee Shop,Café,Italian Restaurant,French Restaurant,Vietnamese Restaurant,Wine Bar,Bar,Restaurant,Bakery
2,南區,38.85,"6,962.68",Hong Kong Island,22.247250,114.158840,0,Cha Chaan Teng,Fast Food Restaurant,Hong Kong Restaurant,Park,Athletics & Sports,BBQ Joint,Chinese Restaurant,Coffee Shop,Dessert Shop,Hotel
3,灣仔區,9.83,"15,300.10",Hong Kong Island,22.279680,114.171680,0,Coffee Shop,Chinese Restaurant,Hotel,Japanese Restaurant,Cha Chaan Teng,Middle Eastern Restaurant,Pizza Place,Clothing Store,Hong Kong Restaurant,Cantonese Restaurant
4,深水埗區,9.35,"41,529.41",Kowloon,22.330740,114.162200,0,Noodle House,Chinese Restaurant,Dessert Shop,Hong Kong Restaurant,Italian Restaurant,Shopping Mall,Snack Place,Electronics Store,Market,Malay Restaurant
6,觀塘區,11.27,"56,779.05",Kowloon,22.313260,114.225810,0,Chinese Restaurant,Café,Coffee Shop,Cha Chaan Teng,Fast Food Restaurant,Japanese Restaurant,Restaurant,Cantonese Restaurant,Sushi Restaurant,Department Store
7,黃大仙區,9.30,"45,645.16",Kowloon,22.333530,114.196860,0,Coffee Shop,Fast Food Restaurant,Noodle House,Asian Restaurant,Hotel,Climbing Gym,Pizza Place,Chinese Restaurant,Diner,Sushi Restaurant
8,油尖旺區,6.99,"44,864.09",Kowloon,22.321380,114.172600,0,Noodle House,Cantonese Restaurant,Cha Chaan Teng,Dessert Shop,Chinese Restaurant,Vegetarian / Vegan Restaurant,Market,Coffee Shop,Beer Bar,Steakhouse


In [24]:
#2nd cluster
hk_merged.loc[hk_merged['Cluster Labels'] == 1, hk_merged.columns[[1] + list(range(3, hk_merged.shape[1]))]]

,Chinese,Area,Density,Region,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,東區,18.56,"31,217.67",Hong Kong Island,22.28411,114.22414,1,Park,Chinese Restaurant,Hong Kong Restaurant,Cantonese Restaurant,Indian Restaurant,French Restaurant,Japanese Restaurant,Fish & Chips Shop,Restaurant,Harbor / Marina


In [25]:
hk_merged.loc[hk_merged['Cluster Labels'] == 2, hk_merged.columns[[1] + list(range(3, hk_merged.shape[1]))]]

,Chinese,Area,Density,Region,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,九龍城區,10.02,"40,194.70",Kowloon,22.3282,114.19155,2,Thai Restaurant,Dessert Shop,Chinese Restaurant,Coffee Shop,Café,Asian Restaurant,Halal Restaurant,Noodle House,Fast Food Restaurant,Cha Chaan Teng
